In [1]:
import os
import sys
import numpy as np
from collections import defaultdict, Counter
import time
from matplotlib import pyplot
import json5
from matplotlib import pyplot as pyplot
from scipy.cluster.hierarchy import dendrogram
import pandas

import importlib
import duhportinf
import duhportinf._grouper
import duhportinf._optimize
import duhportinf._vectorizer
import duhportinf.busdef
import duhportinf.main 
from   duhportinf.busdef import BusDef
from   duhportinf._optimize import MatchCost
from   duhportinf import util

%load_ext autoreload
%autoreload 2

In [2]:
block_root = '/ip-block-designs'
bus_spec_rootdir = '/bus-defs/specs'

In [3]:
def get_ports_from_json5(path):
    with open(path) as fin:
        block = json5.load(fin)
    return util.format_ports(block['definitions']['ports'])

In [4]:
pcie_json_path = os.path.join(
    block_root,
    'block-plda-pcie-controller',
    'PLDA_ws.json5',
)
ddr_json_path = os.path.join(
    block_root,
    'block-cadence-ddr3',
    'ddr.json5',
)
nvdla_json_path = os.path.join(
    block_root,
    'block-nvdla',
    'nvdla.json5',
)
pcie_ports  = get_ports_from_json5(pcie_json_path)
ddr_ports   = get_ports_from_json5(ddr_json_path)
nvdla_ports = get_ports_from_json5(nvdla_json_path)

In [5]:
bus_defs = duhportinf.load_bus_defs(bus_spec_rootdir)

loading 28 bus specs
  - done, loaded 50 bus defs with 535 required and 623 optional ports 


In [6]:
tmp_json_path = os.path.join(block_root, 'tmp', 'tmp.json5')
tmp_ports = get_ports_from_json5(tmp_json_path)

In [16]:
broken_json_path =  os.path.join(block_root, 'broken', 'broken.json5')
broken_ports = get_ports_from_json5(broken_json_path)

pg_bus_pairings = []
nid_cost_map = {}

pg, Z, wire_names = duhportinf.get_port_grouper(broken_ports)
for nid, port_group in pg.get_initial_port_groups():
    print('nid:{}, port group size:{}'.format(nid, len(list(port_group))))
    pg_bus_defs = duhportinf.main._get_lfcost_bus_defs(port_group, bus_defs)[:5]
    l_fcost = pg_bus_defs[0][0]
    pg_bus_pairings.append((nid, l_fcost, port_group, pg_bus_defs))
    nid_cost_map[nid] = l_fcost
    
print('cost map')
for nid, fcost in nid_cost_map.items():
    print('  - ', nid, fcost)
    
optimal_nids = pg.get_optimal_groups(nid_cost_map)    
print('num optimal nids:', len(optimal_nids))
print('  - nids', optimal_nids)
opt_pg_bus_pairings = list(sorted(filter(
    lambda x : (                                        
        # must be on an optimal path for some port                   
        x[0] in optimal_nids and                  
        # at least 4 ports in a group    
        len(x[2]) > 3                                    
    ),                                                                                                                                         
    pg_bus_pairings,                                                                
), key=lambda x: x[1]))   
print('total pre opt bus pairings', len(pg_bus_pairings))
print('post bus pairings', len(opt_pg_bus_pairings))

#opt_pg_bus_pairings = duhportinf.main._get_bus_pairings(pg, bus_defs)
#print('num pairings', len(opt_pg_bus_pairings))


nid:24, port group size:2
nid:21, port group size:3
nid:25, port group size:12
nid:19, port group size:3
nid:23, port group size:9
nid:15, port group size:3
nid:22, port group size:6
nid:17, port group size:3
cost map
  -  24 3.44(n:1.72;w:0;d:0)
  -  21 6.22(n:2.61;w:1;d:0)
  -  25 26.35(n:10.67;w:5;d:0)
  -  19 6.22(n:2.61;w:1;d:0)
  -  23 17.80(n:7.90;w:2;d:0)
  -  15 6.31(n:2.65;w:1;d:0)
  -  22 11.48(n:5.24;w:1;d:0)
  -  17 6.22(n:2.61;w:1;d:0)
num optimal nids: 0
  - nids set()
total pre opt bus pairings 8
post bus pairings 0


In [583]:
# check if splitting port names on digits screws up initial port groups
pg, Z, wn = duhportinf.get_port_grouper(ddr_ports)

In [588]:
dport = ('axi0_ARQOS', 1, 1)
for nid, port_group in pg.get_initial_port_groups():
    if dport in port_group:
        print('dport group size: {}'.format(len(port_group)))
        if len(port_group) < 60:
            for port in port_group:
                print(util.words_from_name(port[0]))

dport group size: 56
['axi', '0', 'awvalid']
['axi', '0', 'wparity', 'en']
['axi', '0', 'rdata', 'parity']
['axi', '0', 'rvalid']
['axi', '0', 'rlast']
['axi', '0', 'awsize']
['axi', '0', 'arvalid']
['axi', '0', 'bparity', 'en']
['axi', '0', 'aclk']
['axi', '0', 'awid']
['axi', '0', 'arapcmd']
['axi', '0', 'wvalid']
['axi', '0', 'arlock']
['axi', '0', 'bid']
['axi', '0', 'awqos']
['axi', '0', 'ar', 'parity', 'en']
['axi', '0', 'arregion']
['axi', '0', 'bresp']
['axi', '0', 'bparity']
['axi', '0', 'wlast']
['axi', '0', 'rresp']
['axi', '0', 'awprot']
['axi', '0', 'wctrl', 'parity']
['axi', '0', 'awcobuf']
['axi', '0', 'arid']
['axi', '0', 'rid']
['axi', '0', 'aw', 'parity']
['axi', '0', 'awapcmd']
['axi', '0', 'awcache', '0']
['axi', '0', 'arqos']
['axi', '0', 'rdata']
['axi', '0', 'awburst']
['axi', '0', 'aresetn']
['axi', '0', 'arsize']
['axi', '0', 'bvalid']
['axi', '0', 'wstrb']
['axi', '0', 'ar', 'parity']
['axi', '0', 'rctrl', 'parity']
['axi', '0', 'wdata']
['axi', '0', 'awaddr']

In [ ]:
df = pandas.DataFrame(Z)
df = df.astype({0:int, 1:int, 3:int})
df.to_csv('linkage-tree.csv', header=False, index=False)
df = pandas.DataFrame(wire_names)
df.to_csv('wire-names.csv', header=False, index=False)
df.head()

In [ ]:
def debug(ports, dport, bus_abstract_names):
    # filter to get debug bus defs
    bus_abstract_names = set(bus_abstract_names)
    dbus_defs = list(filter(
        lambda bd: bd.abstract_type.name in bus_abstract_names,
        bus_defs,
    ))
    
    pg, Z, wire_names = duhportinf._grouper.get_port_grouper(ports)
    pg_bus_pairings = duhportinf.main._get_bus_pairings(pg, dbus_defs)
    # filter only for port_group with dport
    pg_bus_pairings = list(filter(
        lambda x: dport in x[2],
        pg_bus_pairings,
    ))
    pg_bus_mappings = duhportinf.main._get_initial_bus_matches(pg, pg_bus_pairings)
    return list(map(lambda x: x[2:], pg_bus_mappings))
    

In [ ]:
port_path = '/ip-block-designs/tmp/tmp.json5' 
ports = duhportinf.get_ports_from_json5(port_path)
dport = ('front_port_axi4_0_ar_ready', 1, -1)
pg_bus_mappings = debug(ports, dport, ['AXI4_rtl'])

In [ ]:
stime = time.time()

nvdla_pg_bus_mappings = duhportinf.get_bus_matches(nvdla_ports, list(bus_defs))
etime = time.time()
print('total time: {}s'.format(etime-stime))

In [ ]:
util.dump_json_bus_candidates('testy.json', nvdla_json_path, nvdla_pg_bus_mappings)

In [ ]:
stime = time.time()

ddr_pg_bus_mappings = duhportinf.get_bus_matches(ddr_ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

In [ ]:
s_pcie_ports = list(filter(lambda p: 'axi4' not in p[0], pcie_ports))
stime = time.time()
pcie_pg_bus_mappings = duhportinf.get_bus_matches(pcie_ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

In [ ]:
def dump_nonaxi_dendogram(path, ports):
    assert path.endswith('.pdf')
    nonaxi_ports = list(filter(
        lambda p: not p[0].startswith('axi'),
        ports,
    ))
    _, Z, wire_names = abxportinf.get_port_grouper(nonaxi_ports)
    fig, ax = pyplot.subplots(1,1, figsize=(10,40))
    _ = dendrogram(
        Z,
        ax=ax,
        orientation='left',
        labels=wire_names,
    )
    pyplot.savefig(path)


In [ ]:
dump_nonaxi_dendogram('ddr-nonaxi-dendogram.pdf', ddr_ports)
dump_nonaxi_dendogram('pcie-nonaxi-dendogram.pdf', pcie_ports)

In [ ]:
util.dump_json_bus_candidates('ddr-busprop.json', ddr_pg_bus_mappings)
util.dump_json_bus_candidates('pcie-busprop.json', pcie_pg_bus_mappings)